In [1]:
import pandas as pd
from datetime import datetime
#import pm4py
import numpy as np
#from scipy.stats import mannwhitneyu
#import collections
#from statistics import mean
#from statistics import median
from collections import defaultdict
#from Levenshtein import distance

from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn import metrics
#from sklearn.metrics.cluster import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, adjusted_mutual_info_score, silhouette_score

import copy

In [2]:
log1 = pm4py.read.read_xes("PO_BPI_2019\BPI_Challenge_2019_DF1.xes")

C:\Users\la1949\Anaconda3\envs\pm4py\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 5292/5292 [00:25<00:00, 205.89it/s]


In [3]:
log2 = pm4py.read.read_xes("PO_BPI_2019\BPI_Challenge_2019_DF2.xes")

parsing log, completed traces :: 100%|██████████| 16097/16097 [00:23<00:00, 675.51it/s] 


In [4]:
log3 = pm4py.read.read_xes("PO_BPI_2019\BPI_Challenge_2019_DF3.xes")

parsing log, completed traces :: 100%|██████████| 196/196 [00:00<00:00, 651.48it/s]


In [5]:
log4 = pm4py.read.read_xes("PO_BPI_2019\BPI_Challenge_2019_DF4.xes")

parsing log, completed traces :: 100%|██████████| 54/54 [00:00<00:00, 2437.08it/s]


In [46]:
# Initialize LabelEncoder
le = LabelEncoder()

In [47]:
log1['type'] = 'DF1'
log1['case:concept:name'] = 'DF1' + log1['case:concept:name']

log2['type'] = 'DF2' 
log2['case:concept:name'] = 'DF2' + log2['case:concept:name']

log3['type'] = 'DF3' 
log3['case:concept:name'] = 'DF3' + log3['case:concept:name']

log4['type'] = 'DF4' 
log4['case:concept:name'] = 'DF4' + log4['case:concept:name']

In [48]:
log = pd.concat([log1, log2, log3, log4]).reset_index()

In [49]:
# Fit and transform the 'tabName_element' column
log['activity'] = le.fit_transform(log['concept:name'])

In [50]:
#Create trace log
logVar0 = log.groupby(['case:concept:name'])['activity'].apply(list).reset_index()

In [51]:
logType = log.groupby(['case:concept:name'])['type'].apply(list).reset_index()
logType['type'] = logType['type'].apply(lambda x: set(x))
logType['type'] = logType['type'].apply(lambda x: ''.join(x))

In [52]:
logVar0 = pd.merge(logVar0, logType, on='case:concept:name')

In [53]:
logVar0

,case:concept:name,activity,type
0,DF1DF1DF12000000001_00001,"[29, 28, 26, 31, 35, 33, 27, 14, 41, 19, 21, 2...",DF1
1,DF1DF1DF12000000002_00001,"[29, 28, 26, 31, 35, 33, 27, 14, 19, 21, 41, 1...",DF1
2,DF1DF1DF12000000005_00002,"[29, 28, 26, 31, 33, 35, 14, 27]",DF1
3,DF1DF1DF12000000006_00001,"[29, 28, 26, 31, 33, 35, 27, 14, 41, 19, 21, 2...",DF1
4,DF1DF1DF12000000008_00001,"[29, 28, 26, 31, 33, 35, 27, 14, 41, 19, 21, 2...",DF1
...,...,...,...
21634,DF4DF4DF44508060051_00020,"[15, 14, 19, 19, 19, 19, 9, 19]",DF4
21635,DF4DF4DF44508068545_00010,"[15, 14, 9, 9, 18, 9, 9, 19]",DF4
21636,DF4DF4DF44508069068_00010,"[15, 14, 9, 9, 18, 9, 19, 6]",DF4
21637,DF4DF4DF44508073350_00030,"[15, 14, 9, 19, 19, 19, 19, 19, 19, 19]",DF4


In [54]:
#logVar0.loc[logVar0['type'] == 'DF4']

In [55]:
#Derive set of unique trace variants + count
logVar0["c:n_chr"] = logVar0["activity"].apply(lambda x: [chr(int(i)) for i in x])
logVar0["strings"] = logVar0["c:n_chr"].apply(lambda x: ''.join(x))

logVar0["act_str"] = logVar0["activity"].apply(lambda x: str(x))
logVar0['Count'] = logVar0.groupby(['act_str'])['activity'].transform('count')
logVar0 = logVar0.drop_duplicates(subset=['act_str'], keep='first')
logVar0 = logVar0.reset_index(drop=True)
logVar0

,case:concept:name,activity,type,c:n_chr,strings,act_str,Count
0,DF1DF1DF12000000001_00001,"[29, 28, 26, 31, 35, 33, 27, 14, 41, 19, 21, 2...",DF1,"[ , , , , #, !, , , ), , , , \r, !, %]", #!)\r!%,"[29, 28, 26, 31, 35, 33, 27, 14, 41, 19, 21, 2...",2
1,DF1DF1DF12000000002_00001,"[29, 28, 26, 31, 35, 33, 27, 14, 19, 21, 41, 1...",DF1,"[ , , , , #, !, , , , , ), , , , , ...", #!)\r!%,"[29, 28, 26, 31, 35, 33, 27, 14, 19, 21, 41, 1...",1
2,DF1DF1DF12000000005_00002,"[29, 28, 26, 31, 33, 35, 14, 27]",DF1,"[ , , , , !, #, , ]", !#,"[29, 28, 26, 31, 33, 35, 14, 27]",36
3,DF1DF1DF12000000006_00001,"[29, 28, 26, 31, 33, 35, 27, 14, 41, 19, 21, 2...",DF1,"[ , , , , !, #, , , ), , , , \r, !, ]", !#)\r!,"[29, 28, 26, 31, 33, 35, 27, 14, 41, 19, 21, 2...",2
4,DF1DF1DF12000000008_00001,"[29, 28, 26, 31, 33, 35, 27, 14, 41, 19, 21, 2...",DF1,"[ , , , , !, #, , , ), , , , \r]", !#)\r,"[29, 28, 26, 31, 33, 35, 27, 14, 41, 19, 21, 2...",36
...,...,...,...,...,...,...,...
10202,DF3DF3DF34508076162_00010,"[41, 40, 14, 4, 4, 8, 4, 20, 2, 13, 20, 13]",DF3,"[), (, , , , , , , , \r, , \r]",)(\r\r,"[41, 40, 14, 4, 4, 8, 4, 20, 2, 13, 20, 13]",1
10203,DF3DF3DF34508076169_00020,"[41, 4, 4, 14, 4, 20, 13, 4, 4]",DF3,"[), , , , , , \r, , ]",)\r,"[41, 4, 4, 14, 4, 20, 13, 4, 4]",1
10204,DF3DF3DF34508076206_00050,"[41, 4, 4, 4, 4, 14, 4, 20]",DF3,"[), , , , , , , ]",),"[41, 4, 4, 4, 4, 14, 4, 20]",4
10205,DF3DF3DF34508076207_00020,"[4, 4, 14, 4, 20, 41, 4, 4]",DF3,"[, , , , , ), , ]",),"[4, 4, 14, 4, 20, 41, 4, 4]",1


In [56]:
logVar0['type'].value_counts()

DF2    6191
DF1    3913
DF3     103
Name: type, dtype: int64

In [58]:
n=100

logVar1 = logVar0.loc[logVar0['type'] == 'DF1']
logVar1 = logVar1.sample(n)

logVar2 = logVar0.loc[logVar0['type'] == 'DF2']
logVar2 = logVar2.sample(n)

logVar3 = logVar0.loc[logVar0['type'] == 'DF3']
logVar3 = logVar3.sample(n)

In [59]:
logVar = pd.concat([logVar1, logVar2, logVar3]).reset_index()

In [112]:
logVar

,index,case:concept:name,activity,type,c:n_chr,strings,act_str,Count,class,words,length
0,2648,DF1DF1DF14508046320_00001,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 1...",DF1,"[, , , , , , , , , , , , , , , ...",,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 1...",1,0,14 21 21 21 21 21 21 21 21 21 21 19 21 21 21 2...,37
1,2462,DF1DF1DF14507042050_00001,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...",DF1,"[, , , , , , , , , , , , , , , ...",,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...",1,0,14 21 21 21 21 21 21 21 21 21 21 21 21 21 19 1...,42
2,2292,DF1DF1DF14507038748_00030,"[14, 19, 21, 40, 41, 20, 2, 20, 13]",DF1,"[, , , (, ), , , , \r]",()\r,"[14, 19, 21, 40, 41, 20, 2, 20, 13]",1,0,14 19 21 40 41 20 2 20 13,9
3,1690,DF1DF1DF14507025269_00001,"[14, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 2...",DF1,"[, , , , , , , , , , , , , , , ...",...,"[14, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 2...",1,0,14 19 19 19 19 19 19 19 19 19 19 21 21 21 21 2...,311
4,2670,DF1DF1DF14508046610_00001,"[14, 19, 19, 21, 21, 21, 21, 19, 19, 21, 21, 2...",DF1,"[, , , , , , , , , , , , , , , ...",,"[14, 19, 19, 21, 21, 21, 21, 19, 19, 21, 21, 2...",1,0,14 19 19 21 21 21 21 19 19 21 21 21 21 21 19 1...,28
...,...,...,...,...,...,...,...,...,...,...,...
295,10187,DF3DF3DF34507076145_00080,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4,...",DF3,"[, , , , , , , , , , , , , ), , ...",),"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4,...",1,2,4 4 4 4 4 4 4 4 4 4 4 4 4 41 4 14 4 20 4 4,20
296,10169,DF3DF3DF34507076139_00090,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4, 14, 4...",DF3,"[, , , , , , , , , , , ), , , , ...",),"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4, 14, 4...",1,2,4 4 4 4 4 4 4 4 4 4 4 41 4 14 4 20 4 4,18
297,10193,DF3DF3DF34507076150_00140,"[4, 4, 4, 4, 4, 4, 4, 4, 14, 4, 20, 41, 38, 4,...",DF3,"[, , , , , , , , , , , ), &, , , ...",)&,"[4, 4, 4, 4, 4, 4, 4, 4, 14, 4, 20, 41, 38, 4,...",4,2,4 4 4 4 4 4 4 4 14 4 20 41 38 4 4 4 4,17
298,10201,DF3DF3DF34508076157_00030,"[4, 4, 4, 41, 4, 4, 14, 20, 4, 4]",DF3,"[, , , ), , , , , , ]",),"[4, 4, 4, 41, 4, 4, 14, 20, 4, 4]",1,2,4 4 4 41 4 4 14 20 4 4,10


In [61]:
logVar['type'].value_counts()

DF1    100
DF2    100
DF3    100
Name: type, dtype: int64

In [80]:
#logVar.to_csv('logVar_PO_BPI2019_Sample.csv', index = False)

In [2]:
#To be able to replicate our results we proivde the randomized sample use for the calculations
logVar = pd.read_csv('logVar_PO_BPI2019_Sample.csv')
logVar

,index,case:concept:name,activity,type,c:n_chr,strings,act_str,Count,class
0,2648,DF1DF1DF14508046320_00001,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 1...",DF1,"['\x0e', '\x15', '\x15', '\x15', '\x15', '\x15...",,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 1...",1,0
1,2462,DF1DF1DF14507042050_00001,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...",DF1,"['\x0e', '\x15', '\x15', '\x15', '\x15', '\x15...",,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...",1,0
2,2292,DF1DF1DF14507038748_00030,"[14, 19, 21, 40, 41, 20, 2, 20, 13]",DF1,"['\x0e', '\x13', '\x15', '(', ')', '\x14', '\x...",()\r,"[14, 19, 21, 40, 41, 20, 2, 20, 13]",1,0
3,1690,DF1DF1DF14507025269_00001,"[14, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 2...",DF1,"['\x0e', '\x13', '\x13', '\x13', '\x13', '\x13...",...,"[14, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 2...",1,0
4,2670,DF1DF1DF14508046610_00001,"[14, 19, 19, 21, 21, 21, 21, 19, 19, 21, 21, 2...",DF1,"['\x0e', '\x13', '\x13', '\x15', '\x15', '\x15...",,"[14, 19, 19, 21, 21, 21, 21, 19, 19, 21, 21, 2...",1,0
...,...,...,...,...,...,...,...,...,...
295,10187,DF3DF3DF34507076145_00080,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4,...",DF3,"['\x04', '\x04', '\x04', '\x04', '\x04', '\x04...",),"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4,...",1,2
296,10169,DF3DF3DF34507076139_00090,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4, 14, 4...",DF3,"['\x04', '\x04', '\x04', '\x04', '\x04', '\x04...",),"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4, 14, 4...",1,2
297,10193,DF3DF3DF34507076150_00140,"[4, 4, 4, 4, 4, 4, 4, 4, 14, 4, 20, 41, 38, 4,...",DF3,"['\x04', '\x04', '\x04', '\x04', '\x04', '\x04...",)&,"[4, 4, 4, 4, 4, 4, 4, 4, 14, 4, 20, 41, 38, 4,...",4,2
298,10201,DF3DF3DF34508076157_00030,"[4, 4, 4, 41, 4, 4, 14, 20, 4, 4]",DF3,"['\x04', '\x04', '\x04', ')', '\x04', '\x04', ...",),"[4, 4, 4, 41, 4, 4, 14, 20, 4, 4]",1,2


In [3]:
import ast

logVar["activity"] = logVar["activity"].apply(lambda x: ast.literal_eval(x))
logVar["c:n_chr"] = logVar["activity"].apply(lambda x: [chr(int(i)) for i in x])
logVar["strings"] = logVar["c:n_chr"].apply(lambda x: ''.join(x))

In [4]:
logVar["length"] = logVar["activity"].apply(lambda x: len(x))

In [5]:
logVar['length'].max()

324

In [6]:
logVar['length'].min()

8

## Ground truth 

In [7]:
#Create dictionary with true labels

labelDict1 = logVar['class'].to_dict()
#labelDict1

In [8]:
logVar["c:n_chr"] = logVar["activity"].apply(lambda x: [chr(i) for i in x])
logVar["strings"] = logVar["c:n_chr"].apply(lambda x: ''.join(x))

In [9]:
def matrix_calc(features, distance):
    n = len(features)
    dist_matrix = np.zeros((n,n))
    
    for i in range(n):
        for j in range(i, n):
            dist_matrix[i,j] = distance(features[i], features[j])
            dist_matrix[j,i] = dist_matrix[i,j]
    
    return dist_matrix

In [10]:
def results(DistMatrix):
    print('NN:   ' + str(NearestNeighbor(DistMatrix, labelDict1)))
    print('P@10: ' + str(PrecisionAtK(DistMatrix, labelDict1, 10)))

    triplet = BatchAllTtripletLoss()
    print('Tri:  ' + str(triplet.forward(DistMatrix,labelDict1)))
    print('Sil:  ' + str(Silhouette(DistMatrix, labelDict1)))

## Evaluation based on Nearest Neighbor

In [11]:
def NearestNeighbor(matrix, labelDict):
    
    clusterLabel = set(labelDict.values())
    clusterMeasuredCount = dict.fromkeys(list(clusterLabel), 0)
    
    for i in range(len(matrix)):
        #delete 0 in array matrix[i] for distance between (i,i)
        x = np.delete(matrix[i], i) 
        #identify min distance/value in array
        y = min(x)
        
        #identify position of y AND select first position/pair appearing in the array in case there are muliple pairs with identical min distance
        nearestNeighbor = np.where(x == y)[0] #problem if multiple positions??
        nearestNeighbor = int(nearestNeighbor[0])
        
        #true Label for i
        trueLabel = labelDict[i]
        
        #for label comparison add +1 to dict position if position occurs after i (because of the deletion of 0 at the beginning)
        if nearestNeighbor >= i:
            if trueLabel == labelDict[nearestNeighbor + 1]:
                clusterMeasuredCount[trueLabel] += 1         
        else:
            if trueLabel == labelDict[nearestNeighbor]:
                clusterMeasuredCount[trueLabel] += 1
    
    #Count the (true) number of traces per label/attribute
    clusterTrueCount = {}
    for i in clusterLabel:
        clusterTrueCount[i] = list(labelDict.values()).count(i)
    
    #Divide number of nearest neighbours with identical label by the respective (true number) of traces with this label
    metric = 0
    for i in clusterLabel:
        metric += clusterMeasuredCount[i] / clusterTrueCount[i]
        
    #print(clusterMeasuredCount, clusterTrueCount, metric)
    
    #print(clusterMeasuredCount)
    #print(clusterTrueCount)
    #print(clusterLabel)
    #print(metric)
    return metric / len(clusterLabel)


'''
x = np.array([[ 0, 12,  8, 10, 12],
       [5,  0,  9, 14, 17],
       [ 8,  9,  0, 12, 10],
       [ 8,  9,  1, 0, 10],
       [ 8,  9,  5, 4, 0]])

labelDict2 = {0:1,1:2,2:1,3:3,4:3}
NearestNeighbor(x, labelDict2)
'''

'\nx = np.array([[ 0, 12,  8, 10, 12],\n       [5,  0,  9, 14, 17],\n       [ 8,  9,  0, 12, 10],\n       [ 8,  9,  1, 0, 10],\n       [ 8,  9,  5, 4, 0]])\n\nlabelDict2 = {0:1,1:2,2:1,3:3,4:3}\nNearestNeighbor(x, labelDict2)\n'

## Evaluation based on Precision@k

In [12]:
def get_key_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    #return None  # Return None if the value is not found in the dictionary




def PrecisionAtK(matrix, labelDict, k):
    
    clusterLabel = set(labelDict.values())
    clusterMeasuredCount = dict.fromkeys(list(clusterLabel), 0)
    
    
    for i in range(len(matrix)):
        
        #true Label for i
        trueLabel = labelDict[i]

        
        #delete 0 in array matrix[i] for distance between (i,i)   
        x = np.delete(matrix[i], i)
        
        #get all k minimum values
        nnValues = []
        for l in range(k):
            y = min(x)
            nnValues.append(y)
            x = np.delete(x, np.where(x == y)[0][0])
        
        
        #get location of minimum values
        z = np.delete(matrix[i], i)
        
        #create dict from array with {position:value}
        my_dict = {}
        for m in range(len(z)):
            my_dict[m] = z[m]
        
        key_list = []
        for n in nnValues:
            key = get_key_by_value(my_dict, n)
            key_list.append(key)
            del my_dict[key]
        
        for o in range(k):
            position = list(key_list)[o]

            if position >= i:
                if trueLabel == labelDict[position + 1]:
                    clusterMeasuredCount[trueLabel] += 1

            else:
                if trueLabel == labelDict[position]:
                    clusterMeasuredCount[trueLabel] += 1

            #transform array to dict
            #get key value from dict
            #compare label
            #remove key+value from dict

            #Need exception in case y == 0 ??
            
    
    #Count the (true) number of traces per label/attribute
    clusterTrueCount = {}
    for i in clusterLabel:
        clusterTrueCount[i] = list(labelDict.values()).count(i)
    
    #Divide number of nearest neighbours with identical label by the respective (true number) of traces with this label
    metric = 0
    for i in clusterLabel:
        metric += clusterMeasuredCount[i] / k / clusterTrueCount[i]
        
    #print(clusterMeasuredCount, clusterTrueCount, metric)
    
    #print(clusterLabel)
    #print(metric)
    return metric / len(clusterLabel)


'''
x3 = np.array([[ 0, 12,  8, 10, 12],
       [5,  0,  9, 14, 17],
       [ 8,  9,  0, 12, 10],
       [ 8,  9,  1, 0, 10],
       [ 4,  8,  4, 4, 0]]) # --> Issue: How to select NN when identical distance values !!!

labelDict3 = {0:1,1:2,2:1,3:3,4:3}
PrecisionAtK(x3, labelDict3, 2)
'''

'\nx3 = np.array([[ 0, 12,  8, 10, 12],\n       [5,  0,  9, 14, 17],\n       [ 8,  9,  0, 12, 10],\n       [ 8,  9,  1, 0, 10],\n       [ 4,  8,  4, 4, 0]]) # --> Issue: How to select NN when identical distance values !!!\n\nlabelDict3 = {0:1,1:2,2:1,3:3,4:3}\nPrecisionAtK(x3, labelDict3, 2)\n'

## Triplet

see: https://towardsdatascience.com/triplet-loss-advanced-intro-49a07b7d8905

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [14]:
def get_triplet_mask(labels):
    
    # step 1 - get a mask for distinct indices
    ###print('labels', labels)
    # shape: (batch_size, batch_size)
    indices_equal = torch.eye(labels.size()[0], dtype=torch.bool, device=labels.device)
    ###print('equal', indices_equal)
    indices_not_equal = torch.logical_not(indices_equal)
    ###print('not_equal', indices_not_equal)

    # shape: (batch_size, batch_size, 1)
    i_not_equal_j = indices_not_equal.unsqueeze(2)
    ###print('i_not_j - unsqueeze2', i_not_equal_j)
    # shape: (batch_size, 1, batch_size)
    i_not_equal_k = indices_not_equal.unsqueeze(1)
    ###print('i_not_k - unsqueeze1', i_not_equal_k)
    # shape: (1, batch_size, batch_size)
    j_not_equal_k = indices_not_equal.unsqueeze(0)
    ###print('j_not_k - unsqueeze0', i_not_equal_k)
    # Shape: (batch_size, batch_size, batch_size)
    distinct_indices = torch.logical_and(torch.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)
    ###print('distinct!!!!', distinct_indices)

    # step 2 - get a mask for valid anchor-positive-negative triplets
    # shape: (batch_size, batch_size)
    labels_equal = labels.unsqueeze(0) == labels.unsqueeze(1)
    ###print('labels_equal', labels_equal)
    # shape: (batch_size, batch_size, 1)
    i_equal_j = labels_equal.unsqueeze(2)
    ###print('i_equal_j', i_equal_j)
    # shape: (batch_size, 1, batch_size)
    i_equal_k = labels_equal.unsqueeze(1)
    ###print('i_equal_k', i_equal_k)
    # shape: (batch_size, batch_size, batch_size)
    valid_indices = torch.logical_and(i_equal_j, torch.logical_not(i_equal_k))
    ###print('valid_indices!!!', valid_indices)
    

    # step 3 - combine two masks
    mask = torch.logical_and(distinct_indices, valid_indices)
    ###print('mask!!', mask)
    return mask

    """compute a mask for valid triplets
    Args:
        labels: Batch of integer labels. shape: (batch_size,)
    Returns:
        Mask tensor to indicate which triplets are actually valid. Shape: (batch_size, batch_size, batch_size)
        A triplet is valid if:
        `labels[i] == labels[j] and labels[i] != labels[k]`
        and `i`, `j`, `k` are different.
    """
    
class custom_activation(nn.Module):
    def __init__(self):
        super(custom_activation, self).__init__()
    
    def forward(self, x):
        x[x>0] = 1
        x[x<=0] = 0
        return x


class BatchAllTtripletLoss(nn.Module):
  """Uses all valid triplets to compute Triplet loss

  Args:
    margin: Margin value in the Triplet Loss equation
  """
  def __init__(self, margin=0): #default margin = 0
    super().__init__()
    self.margin = margin
    self.relu = nn.ReLU() #new
    self.custom = custom_activation()
    
  def forward(self, distance_matrix, labels):
    """computes loss value.

    Args:
      embeddings: Batch of embeddings, e.g., output of the encoder. shape: (batch_size, embedding_dim)
      labels: Batch of integer labels associated with embeddings. shape: (batch_size,)

    Returns:
      Scalar loss value.
    """
    # step 1 - convert to tensor format
    distance_matrix = torch.tensor(distance_matrix)
    labels = torch.tensor(list(labels.values()))


    # step 2 - compute loss values for all triplets by applying broadcasting to distance matrix

    # shape: (batch_size, batch_size, 1)
    anchor_positive_dists = distance_matrix.unsqueeze(2)
    # shape: (batch_size, 1, batch_size)
    anchor_negative_dists = distance_matrix.unsqueeze(1)
    # get loss values for all possible n^3 triplets
    # shape: (batch_size, batch_size, batch_size)
    triplet_loss = anchor_negative_dists - anchor_positive_dists + self.margin
    ###print('tl0',triplet_loss)

    # step 3 - filter out invalid or easy triplets by setting their loss values to 0

    # shape: (batch_size, batch_size, batch_size)
    mask = get_triplet_mask(labels)
    ###print('mask', mask)
    triplet_loss *= mask
    ###print(triplet_loss)
    ###print('tl1:', triplet_loss)
    # easy triplets have negative loss values
    
    triplet_loss = self.custom(triplet_loss)
    ###print(triplet_loss)
    #triplet_loss = F.relu(triplet_loss)

    # step 4 - compute scalar loss value by averaging positive losses
    
    triLossNonZero = (triplet_loss != 0).nonzero(as_tuple=True)
    labelTorchUnique = torch.unique(labels, return_counts=True)
    
    nonZero = len(triLossNonZero[0])
    triLossSum = []
    for i in range(nonZero):
        #Identify L_a --> In Class
        t1 = triLossNonZero[0][i]
        labelIn = labels[t1]
        positionIn = int((labelTorchUnique[0] == labelIn).nonzero(as_tuple=False))
        countIn = labelTorchUnique[1][positionIn]

        #Identify L_b --> Out Class
        t3 = triLossNonZero[2][i]
        labelOut = labels[t3]
        positionOut = int((labelTorchUnique[0] == labelOut).nonzero(as_tuple=False))
        countOut = labelTorchUnique[1][positionOut]

        #Calculate loss
        value = (1/countIn)*(1/countIn)*(1/countOut)  
        ###print(countIn)
        ###print(countOut)
        triLossSum.append(value)
    
    #finally divide by |A|^2-|A|
    A = len(labelTorchUnique[0])  
    lossValue = sum(triLossSum) / (A*A-A)
        
    #OLD
    #E_triplet = (1 / (A^2 - A)) *
    #num_positive_losses = (triplet_loss > eps).float().sum()
    #print(num_positive_losses)
    #print(triplet_loss.sum())
    #triplet_loss = triplet_loss.sum() / (num_positive_losses + eps)
    

    return lossValue

## Silhouette

In [15]:
from sklearn import metrics

def Silhouette(distMatrix, labelDict):
    labelDictList = list(labelDict.values())
    return metrics.silhouette_score(distMatrix, labelDictList)

In [18]:
logVar

,index,case:concept:name,activity,type,c:n_chr,strings,act_str,Count,class,length
0,2648,DF1DF1DF14508046320_00001,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 1...",DF1,"[, , , , , , , , , , , , , , , ...",,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 1...",1,0,37
1,2462,DF1DF1DF14507042050_00001,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...",DF1,"[, , , , , , , , , , , , , , , ...",,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...",1,0,42
2,2292,DF1DF1DF14507038748_00030,"[14, 19, 21, 40, 41, 20, 2, 20, 13]",DF1,"[, , , (, ), , , , \r]",()\r,"[14, 19, 21, 40, 41, 20, 2, 20, 13]",1,0,9
3,1690,DF1DF1DF14507025269_00001,"[14, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 2...",DF1,"[, , , , , , , , , , , , , , , ...",...,"[14, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 2...",1,0,311
4,2670,DF1DF1DF14508046610_00001,"[14, 19, 19, 21, 21, 21, 21, 19, 19, 21, 21, 2...",DF1,"[, , , , , , , , , , , , , , , ...",,"[14, 19, 19, 21, 21, 21, 21, 19, 19, 21, 21, 2...",1,0,28
...,...,...,...,...,...,...,...,...,...,...
295,10187,DF3DF3DF34507076145_00080,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4,...",DF3,"[, , , , , , , , , , , , , ), , ...",),"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4,...",1,2,20
296,10169,DF3DF3DF34507076139_00090,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4, 14, 4...",DF3,"[, , , , , , , , , , , ), , , , ...",),"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4, 14, 4...",1,2,18
297,10193,DF3DF3DF34507076150_00140,"[4, 4, 4, 4, 4, 4, 4, 4, 14, 4, 20, 41, 38, 4,...",DF3,"[, , , , , , , , , , , ), &, , , ...",)&,"[4, 4, 4, 4, 4, 4, 4, 4, 14, 4, 20, 41, 38, 4,...",4,2,17
298,10201,DF3DF3DF34508076157_00030,"[4, 4, 4, 41, 4, 4, 14, 20, 4, 4]",DF3,"[, , , ), , , , , , ]",),"[4, 4, 4, 41, 4, 4, 14, 20, 4, 4]",1,2,10


### Levenshtein Distance

In [12]:
#import ast

#Calculate Levenshtein Distances
#logVar["activity"] = logVar["activity"].apply(lambda x: ast.literal_eval(x))
#logVar["c:n_chr"] = logVar["activity"].apply(lambda x: [chr(int(i)) for i in x])
#logVar["strings"] = logVar["c:n_chr"].apply(lambda x: ''.join(x))

In [89]:
#Levenshtein Distance
from Levenshtein import distance

List = list(logVar["strings"])

dist_matrix = np.zeros((len(List),len(List)),dtype=int)

for i in range(0,len(List)):
    for j in range(0,len(List)):
        dist_matrix[i,j] = distance(List[i],List[j])

lev_dis = dist_matrix
lev_dis

array([[ 0,  9, 34, ..., 37, 37, 37],
       [ 9,  0, 39, ..., 42, 42, 42],
       [34, 39,  0, ..., 16,  9,  9],
       ...,
       [37, 42, 16, ...,  0,  8, 11],
       [37, 42,  9, ...,  8,  0,  5],
       [37, 42,  9, ..., 11,  5,  0]])

In [90]:
Matrix = lev_dis

print('NN:   ' + str(NearestNeighbor(Matrix, labelDict1)))
print('P@10: ' + str(PrecisionAtK(Matrix, labelDict1, 10)))

triplet = BatchAllTtripletLoss()
print('Tri:  ' + str(triplet.forward(Matrix,labelDict1)))
print('Sil:  ' + str(Silhouette(Matrix, labelDict1)))

NN:   0.8733333333333334
P@10: 0.832
Tri:  tensor(0.7276)
Sil:  0.12470177849800047


### Normalized Levenshtein Distance

In [91]:
List = logVar["strings"]

n = len(List)
dist_matrix = np.zeros((n,n))    # initialize distance matrix to a square of zeros

for i in range(n):
    for j in range(i, n):
        dist_matrix[i,j] = distance(List[i], List[j]) / max(len(List[i]),len(List[j]))
        dist_matrix[j,i] = dist_matrix[i,j]       # for the symmetric part, no computation

lev_dis_norm = dist_matrix
lev_dis_norm

array([[0.        , 0.21428571, 0.91891892, ..., 1.        , 1.        ,
        1.        ],
       [0.21428571, 0.        , 0.92857143, ..., 1.        , 1.        ,
        1.        ],
       [0.91891892, 0.92857143, 0.        , ..., 0.94117647, 0.9       ,
        1.        ],
       ...,
       [1.        , 1.        , 0.94117647, ..., 0.        , 0.47058824,
        0.64705882],
       [1.        , 1.        , 0.9       , ..., 0.47058824, 0.        ,
        0.5       ],
       [1.        , 1.        , 1.        , ..., 0.64705882, 0.5       ,
        0.        ]])

In [92]:
Matrix = lev_dis_norm

print('NN:   ' + str(NearestNeighbor(Matrix, labelDict1)))
print('P@10: ' + str(PrecisionAtK(Matrix, labelDict1, 10)))

triplet = BatchAllTtripletLoss()
print('Tri:  ' + str(triplet.forward(Matrix,labelDict1)))
print('Sil:  ' + str(Silhouette(Matrix, labelDict1)))

NN:   0.91
P@10: 0.8496666666666667
Tri:  tensor(0.8249)
Sil:  0.3633665937907341


### Cosine based on 1-gram

In [17]:
#Create 1-gram

def createVector(charList):
    #dtype = [('structure', 'S10'), ('relfrequ', float)]
    arrayList = np.array(charList)
    unique, counts = np.unique(arrayList, return_counts=True)
    #calculate relative frequency
    relFrequList = np.array((unique, counts)).T
    uniqueList = list(unique)
    return relFrequList[relFrequList[:, 0].argsort()]
    #check completeness
    #if 'tree' not in uniqueList:
        #relFrequList = np.append(relFrequList, np.array([['tree', 0]]), axis=0)
        #print(relFrequList)

        
#Change data format from string to list of unique characters
logVar["1-gram"] = logVar["c:n_chr"].apply(lambda x: createVector(tuple(x)))
#logVar


In [17]:
# Euclidean distance Normalized 1-gram
# see https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy

def alignArrays(array1, array2):
    commonSet = set(array1[:,0]).union(array2[:,0])
    
    for i in commonSet:
        if i not in array1[:,0]:
            array1 = np.append(array1, np.array([[i, '0']]), axis=0)
        if i not in array2[:,0]:
            array2 = np.append(array2, np.array([[i, '0']]), axis=0)
    return array1[array1[:, 0].argsort()], array2[array2[:, 0].argsort()]

In [18]:
from scipy.spatial import distance

def cosineDist(frequVector1, frequVector2):
    Vector1, Vector2 = alignArrays(frequVector1, frequVector2)
    a = Vector1[:,1].astype(int)
    b = Vector2[:,1].astype(int)
    dist_matrix = distance.cosine(a, b)
    return dist_matrix

In [23]:
#Cosine distance based on 1-gram

cos1_dis = matrix_calc(logVar["1-gram"],cosineDist)
results(cos1_dis)

NN:   0.89
P@10: 0.8753333333333333
Tri:  tensor(0.8530)
Sil:  0.5236124497291433


### Cosine based on 2-gram

In [18]:
#Change data format from string to list of unique characters
#logVar["charList"] = logVar["trace_variant"].apply(lambda x: list(x))
#logVar

def df_list(list_of_char):
    extList = list_of_char.copy()
    extList.insert(0, '*') 
    extList.append('$')
    list_new = []
    for i in range(len(extList)):
        new = ''.join(extList[i:i+2])
        list_new.append(new)
    del list_new[-1]
    return list_new

#Change data format from string to list of unique characters
logVar["dfList"] = logVar["c:n_chr"].apply(lambda x: df_list(x))
#logVar

logVar["2-gram"] = logVar["dfList"].apply(lambda x: createVector(x))
#logVar

In [25]:
#Cosine distance based on 2-gram

cos2_dis = matrix_calc(logVar["2-gram"],cosineDist)
results(cos2_dis)

NN:   0.9166666666666666
P@10: 0.8763333333333333
Tri:  tensor(0.8366)
Sil:  0.4528433043499739


In [22]:
#Create 3-gram

#Change data format from string to list of unique characters
logVar["charList"] = logVar["c:n_chr"].apply(lambda x: list(x))

def df_list2(list_of_char):
    extList = list_of_char.copy()
    extList.insert(0, '*') 
    extList.append('$')
    list_new = []
    for i in range(len(extList) - 1):
        new = ''.join(extList[i:i+3])
        list_new.append(new)
    del list_new[-1]
    return list_new

In [23]:
logVar["dfList2"] = logVar["charList"].apply(lambda x: df_list2(x))
logVar["3-gram"] = logVar["dfList2"].apply(lambda x: createVector(x))

In [28]:
#Cosine distance based on 3-gram

cos3_dis = matrix_calc(logVar["3-gram"],cosineDist)
results(cos3_dis)

NN:   0.9
P@10: 0.8770000000000001
Tri:  tensor(0.6360)
Sil:  0.3695476268676781


In [75]:
#Aggregation

aggregate1 = cos1_dis + cos2_dis
results(aggregate1)

NN:   0.9166666666666666
P@10: 0.8806666666666666
Tri:  tensor(0.8567)
Sil:  0.5062917401790524


## Euclidean Distance

In [29]:
# Euclidean distance
# see https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy

def euclidDist(frequVector1, frequVector2):
    Vector1, Vector2 = alignArrays(frequVector1, frequVector2)
    a = Vector1[:,1].astype(float)
    b = Vector2[:,1].astype(float)
    euclidean_dist = np.linalg.norm(a-b)
    return euclidean_dist

In [30]:
#Euclidean distance based on 1-gram

euc1_dis = matrix_calc(logVar["1-gram"],euclidDist)
results(euc1_dis)

NN:   0.86
P@10: 0.8566666666666668
Tri:  tensor(0.7468)
Sil:  0.17411620655277507


In [31]:
#Euclidean distance based on 2-gram

euc2_dis = matrix_calc(logVar["2-gram"],euclidDist)
results(euc2_dis)

NN:   0.91
P@10: 0.859
Tri:  tensor(0.7123)
Sil:  0.07853166307706919


In [32]:
#Euclidean distance based on 3-gram

euc3_dis = matrix_calc(logVar["3-gram"],euclidDist)
results(euc3_dis)

NN:   0.89
P@10: 0.848
Tri:  tensor(0.6660)
Sil:  0.004256488460294499


In [33]:
#Aggregation

agg_euc = euc1_dis + euc2_dis
results(agg_euc)

NN:   0.91
P@10: 0.8666666666666667
Tri:  tensor(0.7403)
Sil:  0.13798733142372635


### Jaccard based on 1-gram

In [16]:
#Distance based on activity type

def jaccard_similarity(list1, list2):
    try:
        s1, s2 = set(list1), set(list2)
        return 1 - len(s1 & s2) / len(s1 | s2)
    except ZeroDivisionError:
        return 0

In [35]:
#Jaccard based on 1-gram
Jacc1_dis = matrix_calc(logVar["charList"],jaccard_similarity)
results(Jacc1_dis)

NN:   0.8633333333333333
P@10: 0.8643333333333333
Tri:  tensor(0.7985)
Sil:  0.39786830393716816


In [36]:
#Jaccard based on 2-gram
Jacc2_dis = matrix_calc(logVar["dfList"],jaccard_similarity)
results(Jacc2_dis)

NN:   0.91
P@10: 0.8753333333333333
Tri:  tensor(0.8265)
Sil:  0.31021626233754385


In [37]:
#Jaccard based on 3-gram
Jacc3_dis = matrix_calc(logVar["dfList2"],jaccard_similarity)
results(Jacc3_dis)

NN:   0.9199999999999999
P@10: 0.8803333333333333
Tri:  tensor(0.6353)
Sil:  0.22636820900186685


In [38]:
#Aggregation

agg_jacc1 = Jacc1_dis + Jacc2_dis
results(agg_jacc1)

NN:   0.9333333333333332
P@10: 0.8803333333333333
Tri:  tensor(0.8391)
Sil:  0.38010942529161884


In [39]:
agg_jacc2 = Jacc1_dis + Jacc2_dis + Jacc3_dis
results(agg_jacc2)

NN:   0.9266666666666667
P@10: 0.884
Tri:  tensor(0.8422)
Sil:  0.3500798473718072


In [22]:
logVar

,index,case:concept:name,activity,type,c:n_chr,strings,act_str,Count,class,length,1-gram,dfList,2-gram,charList,dfList2,3-gram
0,2648,DF1DF1DF14508046320_00001,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 1...",DF1,"[, , , , , , , , , , , , , , , ...",,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 1...",1,0,37,"[[, 1], [, 4], [, 32]]","[*, , , , , , , , , , , ...","[[, 1], [, 1], [, 2], [$, 1], [, 3], ...","[, , , , , , , , , , , , , , , ...","[*, , , , , , , , , ...","[[, 1], [$, 1], [, 2], [, 1], [,..."
1,2462,DF1DF1DF14507042050_00001,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...",DF1,"[, , , , , , , , , , , , , , , ...",,"[14, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...",1,0,42,"[[, 1], [, 9], [, 32]]","[*, , , , , , , , , , , ...","[[, 1], [, 6], [, 2], [$, 1], [, 3], ...","[, , , , , , , , , , , , , , , ...","[*, , , , , , , , , ...","[[, 1], [, 3], [, 2], [$, 1], [,..."
2,2292,DF1DF1DF14507038748_00030,"[14, 19, 21, 40, 41, 20, 2, 20, 13]",DF1,"[, , , (, ), , , , \r]",()\r,"[14, 19, 21, 40, 41, 20, 2, 20, 13]",1,0,9,"[[, 1], [\r, 1], [, 1], [, 1], [, 2], [, ...","[*, , , (, (), ), , , \r, \r$]","[[, 1], [\r$, 1], [, 1], [, 1], [, 1],...","[, , , (, ), , , , \r]","[*, , (, (), (), ), , \r, \r$]","[[\r, 1], [, 1], [(, 1], [, 1], [\r..."
3,1690,DF1DF1DF14507025269_00001,"[14, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 2...",DF1,"[, , , , , , , , , , , , , , , ...",...,"[14, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 2...",1,0,311,"[[\r, 1], [, 1], [, 149], [, 1], [, 158], ...","[*, , , , , , , , , , , ...","[[\r$, 1], [, 1], [, 123], [, 26], [\r,...","[, , , , , , , , , , , , , , , ...","[*, , , , , , , , , ...","[[, 1], [, 98], [, 25], [, 26], [..."
4,2670,DF1DF1DF14508046610_00001,"[14, 19, 19, 21, 21, 21, 21, 19, 19, 21, 21, 2...",DF1,"[, , , , , , , , , , , , , , , ...",,"[14, 19, 19, 21, 21, 21, 21, 19, 19, 21, 21, 2...",1,0,28,"[[, 1], [, 11], [, 16]]","[*, , , , , , , , , , , ...","[[, 1], [, 7], [, 4], [, 3], [, 12],...","[, , , , , , , , , , , , , , , ...","[*, , , , , , , , , ...","[[, 1], [, 3], [, 4], [, 4], [,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,10187,DF3DF3DF34507076145_00080,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4,...",DF3,"[, , , , , , , , , , , , , ), , ...",),"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4,...",1,2,20,"[[, 17], [, 1], [, 1], [), 1]]","[*, , , , , , , , , , , ...","[[, 13], [, 1], [, 1], [$, 1], [), 1],...","[, , , , , , , , , , , , , ), , ...","[*, , , , , , , , , ...","[[, 11], [$, 1], [), 1], [, 1], [..."
296,10169,DF3DF3DF34507076139_00090,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4, 14, 4...",DF3,"[, , , , , , , , , , , ), , , , ...",),"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 41, 4, 14, 4...",1,2,18,"[[, 15], [, 1], [, 1], [), 1]]","[*, , , , , , , , , , , ...","[[, 11], [, 1], [, 1], [$, 1], [), 1],...","[, , , , , , , , , , , ), , , , ...","[*, , , , , , , , , ...","[[, 9], [$, 1], [), 1], [, 1], [,..."
297,10193,DF3DF3DF34507076150_00140,"[4, 4, 4, 4, 4, 4, 4, 4, 14, 4, 20, 41, 38, 4,...",DF3,"[, , , , , , , , , , , ), &, , , ...",)&,"[4, 4, 4, 4, 4, 4, 4, 4

## Parallelism based on ordering of activities

In [19]:
# Get set of unique df-relations

# Step 1: Drop None values (if any)
df_clean = logVar.dropna(subset=['dfList'])

# Step 2: Explode the lists into individual rows
exploded = df_clean.explode('dfList')

# Step 3: Get unique values
unique_values = exploded['dfList'].unique()

# Step 4: Convert to set (optional)
df_relations = set(unique_values)

#df_relations


In [24]:
# Get set of unique 3-grams

# Step 1: Drop None values (if any)
df_clean = logVar.dropna(subset=['dfList2'])

# Step 2: Explode the lists into individual rows
exploded = df_clean.explode('dfList2')

# Step 3: Get unique values
unique_values = exploded['dfList2'].unique()

# Step 4: Convert to set (optional)
gram_relations = set(unique_values)

#gram_relations

In [25]:
def join_and_replace(lst, start, end):
    # Join the specified slice
    joined = ''.join(sorted(lst[start:start+end])) # sort according to alphabet
    # Replace that slice with the joined string
    return lst[:start] + [joined] + lst[start+end:]

def checkPara(traceList,dfSet,gramSet):
    i = 0
    while i < len(traceList) - 1:
        #print('i=',i)
        j = i
        count = 1
        #print('j=',j)
        #print(traceList[j + 1] + traceList[j])
        
        #check whether df-relation exist in opposite direction dfSet or gramSet
        while j < len(traceList) - 1 and traceList[j + 1] + traceList[j] in dfSet:
            if traceList[j] + traceList[j+1] + traceList[j] not in gramSet and traceList[j+1] + traceList[j] + traceList[j+1] not in gramSet:
                #print(traceList[j + 1] + traceList[j])
                j +=1
                count += 1
                #print('count ', count)
            else:
                break
        
        if count > 1:
            traceList = join_and_replace(traceList,i,count)
            #print(traceList)
            #print('i=',i)
            #i = j + count -2
            
            #print(len(traceList))
        i += 1
    return traceList

x = ['O','L','P','O','L','R']
dfRel = {'PL', 'OP', 'RL'}
threeGram = {'OLP', 'LPL','POL', 'OLR'}
            
checkPara(x,dfRel,threeGram)

['O', 'L', 'OP', 'LR']

In [26]:
#Generate for each trace the trace including sets of parallel activties
logVar["paraList"] = logVar["charList"].apply(lambda x: checkPara(x,df_relations,gram_relations))

## Graph based features

In [28]:
# 1. transfer strings (string numbers) to integers --> HERE we need a new encoding!

#logVar = logVar.astype({"trace_variant": str}) #--> only needed if list values were transformed to int at one point

def intEncoder(character_List):
    return [np.where(np.array(list(dict.fromkeys(character_List)))==e)[0][0]for e in character_List]

logVar["intList"] = logVar["paraList"].apply(lambda x: intEncoder(x))


# 2. transfer intList to int_tupleList

#Create tuple lists
def tuple_list(list_of_encodedActivities):
    #list.insert(0, '*')
    #list.append('*')
    list_new = []
    last_element = list_of_encodedActivities[-1]
    for i in range(len(list_of_encodedActivities)):
        new = tuple(list_of_encodedActivities[i:i+2])
        list_new.append(new)
    del list_new[-1]
    if list_of_encodedActivities.count(last_element) == 1: #check wether last activity in trace has some adjancency relation
        list_new.append((last_element,)) ### NOT Correct
    return list_new

#q = [0,0,0,0,1,1,2,3,4,5,3,2,4,0,5,6]
#tuple_list(q)

logVar["int_tupleList"] = logVar["intList"].apply(lambda x: tuple_list(x))
#logVar["int_tupleList"]

# 3. generate Adjacency List

def adj_list(list_of_tuples):
    adj_list_new = {}
    try:
        for node1, node2 in list_of_tuples:
            #print(node1, node2)
            if node1 not in adj_list_new:
                newlist = []
                newlist.append(node2)
                adj_list_new[node1] = newlist
                #print(adj_list3)
        
            else:
                if node2 not in adj_list_new[node1]:
                    #mylist.extend(adj_list3[node1])
                    adj_list_new[node1].append(node2)
                    #print(adj_list3)
                    #adj_list3[node1] = mylist
    
    #in case activity has no adjacent activity - only possible for last activity --> tuple: (lastAct,)
    except ValueError as ve:
        lastValue = list_of_tuples[-1][0] 
        adj_list_new[lastValue] = list()
    return list(adj_list_new.values())



logVar["int_adjList"] = logVar["int_tupleList"].apply(lambda x: adj_list(x))
#logVar["int_adjList"]




In [29]:
from collections import deque

def bfs_4(graph, start, end):
    
    graph = {v: k for v, k in enumerate(graph)}
    #print(start, end)
    queue = deque([(start, 0)])
    seen = set()
    while queue:
        #print(queue)
        node, distance = queue.popleft()
        #if not node:
            #print(start, end, queue)
            #print("GRAPH LIST", graph)
        if node in seen:
            continue
        seen.add(node)
        if node == end:
            return distance 
        for adjacent in graph.get(node, []):
            queue.append((adjacent, distance + 1))
        
#x = {0: [0, 1], 1: [2, 1, 0, int], 2:[2], [3: [1, 5, 3, 7], 4: [3], 5: [6, 5], 6: [1, 7], 7: [8, 9, 7], 8: [5, 8, 10], 9: [3]}
#y = [[0, 1, 5], [1, 2], [3, 4], [4, 2], [5, 0], [3, 6], []]
#bfs_4(y, 1, 6)

In [30]:
from collections import defaultdict, deque

def reverse_graph(graph):
    reversed_graph = defaultdict(list)
    for node in graph:
        for neighbor in graph[node]:
            reversed_graph[neighbor].append(node)
    return reversed_graph


def bfs_5(graph, start, end):
    queue = deque([(start, 0)])
    seen = set()
    visited = {}
    while queue:
        node, distance = queue.popleft()
        if node in seen:
            continue
        seen.add(node)
        if node == end: # maybe quicker if adjacent directly checked
            return visited
        for adjacent in graph.get(node, []):
            queue.append((adjacent, distance + 1))
            if adjacent not in visited:
                visited.update({adjacent:distance})

            
def common_ancestors(graph, node1, node2): 
    #remove cross type edge between node1 and node2
    graph[node1].remove(node2) #maybe issue, if graph is changed and not copy of graph?
    graph = {v: k for v, k in enumerate(graph)}
    graphReverse = reverse_graph(graph)
    setNode1 = bfs_5(graphReverse, node1, 0)
    setNode2 = bfs_5(graphReverse, node2, 0)
    if next((a for a in list(setNode1) if a in list(setNode2)), None) == None:
        firstCommonAnces = next((a for a in list(setNode2) if a in list(setNode1)), None)
    else:
        firstCommonAnces = next((a for a in list(setNode1) if a in list(setNode2)), 0)
    
    #uses a hash map to identify the first common ancestor in both lists
    #looks for the first common ancestor in setNode1, which can also be found in setNode2 
    #--> this might not be the closest distance between setNode1 and setNode2
    #--> e.g., for x= [0,1,3,7,5,6] and y= [4,5,7,8,3] 7 might be closest ancestor, although algo detects 3 !
    #distance = setNode1[firstCommonAnces] + setNode2[firstCommonAnces]
    
    
    if firstCommonAnces != None:   # ISSUE: in some cases the firstCommonAnces cannot be detected!
        ancesDistNode1 =  setNode1[firstCommonAnces] + 1 #the edge from node1 to first parent is counted as 0 by algorithm, therefore +1
        ancesDistNode2 =  setNode2[firstCommonAnces] + 1
        numberSkips = abs(ancesDistNode1 - ancesDistNode2)
        numberCross = min(ancesDistNode1, ancesDistNode2)
    else:
        numberSkips, numberCross = (0,1)
    return numberSkips, numberCross
    #if all(x in crossType for x in i):
    

    

#graphList = [[1], [2, 4, 1], [3, 2, 1], [], [5, 4], [5, 4, 6], [7], []]
#c = [[1, 4], [2], [3], [0, 5], [3, 5], []]
#c2 = {v: k for v, k in enumerate(c)}
#common_ancestors(c, 4, 5)
#reverse_graph(c2)

In [31]:
#Create List for decoding traces
from collections import OrderedDict
logVar["indexList"] = logVar["paraList"].apply(lambda x: list(OrderedDict.fromkeys(x)))

### Cosine Edge Type + length (no df relations)

In [42]:
class Graph1:
    # instance variables
    def __init__(self, graph_list2, indexList):
        # v is the number of nodes/vertices
        self.time = 0
        self.traversal_array = []
        self.structural_array = [['sequ', 1]]
        #self.structural_array = []
        self.graph_list = graph_list2
        self.v = len(graph_list2)
        self.indexList = indexList

    # function for dfs
    def dfs(self):
        self.start_time = [-1]*self.v
        self.end_time = [-1]*self.v
 
        for node in range(self.v):
            if self.start_time[node] == -1:
                self.traverse_dfs(node)
                
        return self.structural_array
        #print()
        #print("DFS Traversal: ", self.traversal_array)
        #print()
 
    def traverse_dfs(self, node):
        self.traversal_array.append(node)
        # get the starting time
        self.start_time[node] = self.time
        self.time += 1
        # traverse through the neighbours
        for neighbour in self.graph_list[node]:

            # when the neighbor was not yet visited
            if self.start_time[neighbour] == -1:                
                self.structural_array[0][1] += 0
                self.traverse_dfs(neighbour)
                
            # otherwise when the neighbour's visit is still ongoing:
            elif self.end_time[neighbour] == -1:
                if node == neighbour:
                    self.structural_array.append(['1back ',1])
                    #self.structural_array.append(['back ',1])
                    #self.structural_array.append([str(1)+'b'])
                
                elif node in self.graph_list[neighbour]:
                    self.structural_array.append(['2back ',2])
                    #self.structural_array.append(['back ',2])
                    #self.structural_array.append(str(2)+'b')
                    
                else:
                    x = bfs_4(self.graph_list, neighbour, node)
                    self.structural_array.append([str(x+1)+'back ',x+1])
                    #self.structural_array.append(['back ',x+1])
                    #self.structural_array.append(str(x+1)+'b')
                
            # otherwise when the neighbour's visit started before the current node's visit:
            elif self.start_time[node] < self.start_time[neighbour]:
                graph_list_copy = copy.deepcopy(self.graph_list)
                graph_list_copy[node].remove(neighbour)
                y = bfs_4(graph_list_copy, node, neighbour)
                self.structural_array.append([str(y-1)+'forward ',y-1])
                #self.structural_array.append(['forward ' ,y-1])
                #self.structural_array.extend((y-1)*['forward']) # -1 to exclude one edge: (A:B,C;B:C;C:[]) ...the dist A --> C is 2 without forward edge, but we are skipping only one activity
                #self.structural_array.append(str(y-1)+'f')
                
            else:
                #Possibly first check, whether two nodes connected by cross-type have identical parent
                numberSkips, numberCross = common_ancestors(self.graph_list, node, neighbour)
                #self.structural_array.append(['forward ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]),numberSkips])
                self.structural_array.append([str(numberCross)+'cross ',numberCross])
                #self.structural_array.append(['cross ' ,numberCross])
                #self.structural_array.append(str(numberSkips)+'f')
                #self.structural_array.append(str(numberCross)+'c')
    
        # Indentation corrected:
        self.end_time[node] = self.time
        self.time += 1


In [43]:
from collections import Counter

def transform_list_of_pairs(pairs):
    return [pair[0] for pair in pairs]



def count_entries(input_list):
    # Count the occurrences of each unique entry in the list
    counter = Counter(input_list)
    
    # Create a NumPy array from the counter dictionary
    result = np.array([[key, count] for key, count in counter.items()], dtype=object)
    
    return result

# Example usage
#input_list = ['sequ', '2back', '2back']
#result = count_entries(input_list)
#print(result)

logVar["int_strucLengthList2"] = logVar.apply(lambda x: Graph1(x.int_adjList, x.indexList).dfs(), axis =1)
#logVar["relFrequVec1"] = logVar["int_strucLengthList2"].apply(lambda x: transform_list_of_pairs(x))
#logVar["relFrequVec1"] = logVar["relFrequVec1"].apply(lambda x: count_entries(x))

In [ ]:
###### Changed set of high-level features

In [56]:
# First generate list of parallel constructs with # of activities involved

a = logVar['paraList'][0]
print(a)


        
def para_features(para_list):
    paraAct = []
    for i in range(len(para_list)):
        if len(para_list[i]) > 1:
            paraAct.append([str(len(para_list[i])) + 'para', len(para_list[i])])
    return paraAct


para_features(a) 

['\x0e', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x13', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x13', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x13', '\x13']


[]

In [57]:
logVar["int_strucLengthList4"] = logVar["paraList"].apply(lambda x: para_features(x))
logVar["int_strucLengthList5"] = logVar["int_strucLengthList2"] + logVar["int_strucLengthList4"]
logVar["int_strucLengthList5"] = logVar["int_strucLengthList5"].apply(lambda x: np.array(x))
logVar["relFrequVec1"] = logVar["int_strucLengthList5"].apply(lambda x: transform_list_of_pairs(x))
logVar["relFrequVec1"] = logVar["relFrequVec1"].apply(lambda x: count_entries(x))

In [ ]:
#New 2

In [58]:
#Cosine distance
cos_graph_dis = matrix_calc(logVar["relFrequVec1"],cosineDist)
results(cos_graph_dis)

NN:   0.5766666666666667
P@10: 0.6433333333333333
Tri:  tensor(0.5727)
Sil:  0.056008983310763726


In [59]:
#cos1_dis = matrix_calc(logVar["1-gram"],cosineDist)
#cos2_dis = matrix_calc(logVar["2-gram"],cosineDist)

#Cosine distance
agg_cos = cos1_dis + cos2_dis + cos_graph_dis
results(agg_cos)

NN:   0.9
P@10: 0.879
Tri:  tensor(0.8174)
Sil:  0.4435747597220416


In [ ]:
# Old results without para

In [46]:
#Cosine distance based on edge types
#cos_graph_dis = matrix_calc(logVar["relFrequVec1"],cosineDist)
#results(cos_graph_dis)

NN:   0.38000000000000006
P@10: 0.5996666666666667
Tri:  tensor(0.5413)
Sil:  0.05344526467741865


In [47]:
#Cosine distance based on edge types
#agg_cos = cos1_dis + cos2_dis + cos_graph_dis
#results(agg_cos)

NN:   0.89
P@10: 0.8713333333333333
Tri:  tensor(0.8276)
Sil:  0.44813578030959716


### Jaccard Edge Type and length + df relation

In [32]:
class Graph2:
    # instance variables
    def __init__(self, graph_list2, indexList):
        # v is the number of nodes/vertices
        self.time = 0
        self.traversal_array = []
        self.structural_array = []
        #self.structural_array = []
        self.graph_list = graph_list2
        self.v = len(graph_list2)
        self.indexList = indexList

    # function for dfs
    def dfs(self):
        self.start_time = [-1]*self.v
        self.end_time = [-1]*self.v
 
        for node in range(self.v):
            if self.start_time[node] == -1:
                self.traverse_dfs(node)
                
        return self.structural_array
        #print()
        #print("DFS Traversal: ", self.traversal_array)
        #print()
 
    def traverse_dfs(self, node):
        self.traversal_array.append(node)
        # get the starting time
        self.start_time[node] = self.time
        self.time += 1
        # traverse through the neighbours
        for neighbour in self.graph_list[node]:

            # when the neighbor was not yet visited
            if self.start_time[neighbour] == -1:                
                #self.structural_array[0][1] += 0
                #self.structural_array.append('tree ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ')
                self.structural_array.append('tree')
                self.traverse_dfs(neighbour)
                
            # otherwise when the neighbour's visit is still ongoing:
            elif self.end_time[neighbour] == -1:
                if node == neighbour:
                    self.structural_array.append('back ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(1))
                    #self.structural_array.append([str(1)+'b'])
                
                elif node in self.graph_list[neighbour]:
                    self.structural_array.append('back ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(2))
                    #self.structural_array.append(str(2)+'b')
                    
                else:
                    x = bfs_4(self.graph_list, neighbour, node)
                    self.structural_array.append('back ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(x+1))
                    #self.structural_array.append(str(x+1)+'b')
                
            # otherwise when the neighbour's visit started before the current node's visit:
            elif self.start_time[node] < self.start_time[neighbour]:
                graph_list_copy = copy.deepcopy(self.graph_list)
                graph_list_copy[node].remove(neighbour)
                y = bfs_4(graph_list_copy, node, neighbour)
                self.structural_array.append('forward ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(y-1))
                #self.structural_array.extend((y-1)*['forward']) # -1 to exclude one edge: (A:B,C;B:C;C:[]) ...the dist A --> C is 2 without forward edge, but we are skipping only one activity
                #self.structural_array.append(str(y-1)+'f')
                
            else:
                #Possibly first check, whether two nodes connected by cross-type have identical parent
                numberSkips, numberCross = common_ancestors(self.graph_list, node, neighbour)
                #self.structural_array.append('forward ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(numberSkips))
                self.structural_array.append('cross ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(numberCross))
                #self.structural_array.append(str(numberSkips)+'f')
                #self.structural_array.append(str(numberCross)+'c')
    
        # Indentation corrected:
        self.end_time[node] = self.time
        self.time += 1



In [33]:
logVar["int_strucLengthList3"] = logVar.apply(lambda x: Graph2(x.int_adjList, x.indexList).dfs(), axis =1)
#logVar

In [34]:
# Generate list of parallel constructs with # of activities involved

a = logVar['paraList'][0]
print(a)


        
def para_features2(para_list):
    paraAct = []
    for i in range(len(para_list)):
        if len(para_list[i]) > 1:
            #paraAct.append('para ' + str(len(para_list[i])))
            paraAct.append('para ' + para_list[i])
    return paraAct


para_features2(a) 

['\x0e', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x13', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x13', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x15', '\x13', '\x13']


[]

In [35]:
logVar["int_strucLengthList6"] = logVar["paraList"].apply(lambda x: para_features2(x))
logVar["int_strucLengthList7"] = logVar["int_strucLengthList3"] + logVar["int_strucLengthList6"]

In [36]:
#Jacc similarity based on edge types

jacc_graph = matrix_calc(logVar["int_strucLengthList7"],jaccard_similarity)
results(jacc_graph)

NN:   0.8833333333333333
P@10: 0.8453333333333332
Tri:  tensor(0.6917)
Sil:  0.2426358871571846


In [38]:
#Jacc sim based on edge types
#Jacc1_dis = matrix_calc(logVar["charList"],jaccard_similarity)
#Jacc2_dis = matrix_calc(logVar["dfList"],jaccard_similarity)

agg_jacc = Jacc1_dis + Jacc2_dis + jacc_graph
results(agg_jacc)

NN:   0.9233333333333333
P@10: 0.896
Tri:  tensor(0.8395)
Sil:  0.35966967708093606


In [ ]:
# OLD results without parallel activities

In [50]:
#Jacc similarity based on edge types

#jacc_graph = matrix_calc(logVar["int_strucLengthList3"],jaccard_similarity)
#results(jacc_graph)

NN:   0.8066666666666666
P@10: 0.8079999999999999
Tri:  tensor(0.7157)
Sil:  0.2647951709784663


In [51]:
#Jacc sim based on edge types
#agg_jacc = Jacc1_dis + Jacc2_dis + jacc_graph
#results(agg_cos)

NN:   0.89
P@10: 0.8713333333333333
Tri:  tensor(0.8276)
Sil:  0.44813578030959716


## Eventually Follows

In [52]:
#Spatial distance between strings


from scipy.spatial import distance


def distanceSpatial(traceString, char1, char2):
    positions_letter1 = [pos for pos, char in enumerate(traceString) if char == char1]
    positions_letter2 = [pos for pos, char in enumerate(traceString) if char == char2]
    
    distList = []
    

    for i in range(len(positions_letter1)):
        for j in range(len(positions_letter2)):
            dist = positions_letter2[j] - positions_letter1[i]
            if dist > 0:
                    #print(dist)
                distList.append(dist)
                    
    
    if not distList: #distList.append(0) #in the case the char1 is after char2 asign dist 0, i.e. char2 cannot be reached from char1
        return 0
    else:
        return 1/min(distList)





def commonDistance(trace1, trace2):
    
    commonSet = set(trace1) & set(trace2)

    commonList = list(commonSet)
    commonList.sort()
    #print(commonList)

    n = len(commonSet)
    dist_matrix1 = np.zeros((n,n))
    dist_matrix2 = np.zeros((n,n))

    for i in range(n):
        for j in range(i, n):
            dist_matrix1[i,j] = distanceSpatial(trace1, commonList[i], commonList[j])
        
    for i in range(n):
        for j in range(i, n):
            dist_matrix2[i,j] = distanceSpatial(trace2, commonList[i], commonList[j])
    
    #print(dist_matrix1, dist_matrix2)
    return distance.cosine(dist_matrix1.ravel(), dist_matrix2.ravel())



#x = 'ABCDEF'
#y = 'ABCDEBCDEBCDEF'
#z = 'ABCDEBCDEF'
#print(dist_matrix)
#distanceSpatial(x, 'A', 'E')
#listVec = logVar["strings"]
#x= listVec[0]
#y= listVec[1]
#commonDistance(x, y)

In [63]:
dist_matrix_evFollows = matrix_calc(logVar["strings"],commonDistance)

In [62]:
dist_matrix_evFollows = np.nan_to_num(dist_matrix_evFollows, nan=0)
agg_evFollows = 0.7*dist_matrix_evFollows + 0.3*cos1_dis 
results(agg_evFollows)

NN:   0.9
P@10: 0.8536666666666667
Tri:  tensor(0.7764)
Sil:  0.3019627879207196


## Maximal Repeat

In [64]:
from suffix_tree import Tree

In [65]:
#tree.maximal_repeats
def maxRepeat(tree):
    mrList=[]
    for C, path in sorted(tree.maximal_repeats()):
        mrList.append(str(path))
    return mrList

#test_tree = Tree({"A": "aaacdcdcbedbccbadbdebdc"})
#maxRepeat(test_tree)

In [66]:
#create vector based on maximal repeats
logVar["mrList"] = logVar["strings"].apply(lambda x: maxRepeat(Tree({"A": x})))
logVar["mrVector"] = logVar["mrList"].apply(lambda x: createVector(tuple(x)))

In [69]:
#Cosine distance based on maxR
#cos_mr = matrix_calc(logVar["mrVector"],cosineDist)
#cos_mr = np.nan_to_num(cos_mr, nan=0)
#results(cos_mr)

In [70]:
#Cosine distance based on maxR
cos_mr = matrix_calc(logVar["mrVector"],cosineDist)
cos_mr = np.nan_to_num(cos_mr, nan=0)
results(cos_mr)

NN:   0.3333333333333333
P@10: 0.299
Tri:  tensor(0.5843)
Sil:  0.29013300709293


In [71]:
#Euclidean distance based on maxR
euc_mr = matrix_calc(logVar["mrVector"],euclidDist)
euc_mr = np.nan_to_num(euc_mr, nan=0)
results(euc_mr)

NN:   0.71
P@10: 0.7566666666666667
Tri:  tensor(0.6538)
Sil:  0.17985429098443936


In [74]:
#Jaccard similarity based on maxR
jacc_mr = matrix_calc(logVar["mrList"],jaccard_similarity)
results(jacc_mr)

NN:   0.73
P@10: 0.7896666666666667
Tri:  tensor(0.5814)
Sil:  0.22825602211419682


## Trace2Vec

In [1]:
from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [55]:
logVar["words"] = logVar["activity"].apply(lambda x: x.strip('[]').replace(',', ''))
logVar["words"][0]

'14 21 21 21 21 21 21 21 21 21 21 19 21 21 21 21 21 21 21 21 19 21 21 21 21 21 21 21 21 21 21 21 21 21 21 19 19'

In [57]:
documents = list(logVar['words'])

In [59]:
# Step 1: Tokenize documents
texts = [
    [word for word in document.lower().split()]
    for document in documents
]

In [77]:
# Step 2: Tag the documents

from gensim.models.doc2vec import TaggedDocument

tagged_docs = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(documents)]

In [104]:
#Derive and evaluate model

from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity

def trainModel(documents,vec_size):
    
    # Step 3: Initialize and train the model
    model = Doc2Vec(vector_size=vec_size, window=3, min_count=0.0, min_alpha=0.025, epochs=10, dm = 0, alpha=0.025)
    model.build_vocab(documents)
    model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)
    
    # Step 4: infer vectors for each document
    vectors = [model.infer_vector(doc) for doc in texts]
    
    # Step 5: calculate cosine distances
    cos = cosine_similarity(vectors)
    
    return results(cos)


In [105]:
trainModel(tagged_docs,16)

NN:   0.4533333333333333
P@10: 0.35600000000000004
Tri:  tensor(0.3173)
Sil:  0.20832829


In [106]:
trainModel(tagged_docs,32)

NN:   0.39666666666666667
P@10: 0.296
Tri:  tensor(0.2923)
Sil:  0.23501979


In [107]:
trainModel(tagged_docs,64)

NN:   0.41333333333333333
P@10: 0.3283333333333333
Tri:  tensor(0.3111)
Sil:  0.22917262


In [108]:
trainModel(tagged_docs,128)

NN:   0.38000000000000006
P@10: 0.31733333333333336
Tri:  tensor(0.3091)
Sil:  0.23530532
